In [0]:
CREATE TABLE IF NOT EXISTS orders_demo (
  order_id        BIGINT,
  customer_id     BIGINT,
  order_date      DATE,
  order_status    STRING,
  order_priority  STRING,
  total_price     DECIMAL(12,2)
);

INSERT INTO orders_demo VALUES
  (1001, 10, DATE'2025-01-05', 'O', '1-URGENT', 120.00),
  (1002, 10, DATE'2025-01-20', 'F', '2-HIGH',   250.00),
  (1003, 11, DATE'2025-02-02', 'P', '3-MED',     90.00),
  (1004, 12, DATE'2025-02-15', 'O', '2-HIGH',   500.00),
  (1005, 12, DATE'2025-03-01', 'F', '5-LOW',     75.50),
  (1006, 13, DATE'2025-03-11', 'O', '4-LOW',    310.00);

In [0]:
CREATE OR REPLACE VIEW orders_metrics_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Demo metric view over orders_demo with materialization"

  source: orders_demo

  dimensions:
    - name: Order_Month
      expr: DATE_TRUNC('MONTH', order_date)

    - name: Order_Status
      expr: CASE
        WHEN order_status = 'O' then 'Open'
        WHEN order_status = 'P' then 'Processing'
        WHEN order_status = 'F' then 'Fulfilled'
        END

  measures:
    - name: Order_Count
      expr: COUNT(1)

    - name: Total_Revenue
      expr: SUM(total_price)

    - name: Total_Revenue_for_Open_Orders
      expr: SUM(total_price) FILTER (WHERE order_status = 'O')

  materialization:
    schedule: EVERY 1 HOUR
    mode: relaxed
    materialized_views:
      - name: Total_Revenue
        type: aggregated
        dimensions:
          - Order_Month
        measures:
          - Total_Revenue
          - Total_Revenue_for_Open_Orders
$$;

In [0]:
SELECT
  DATE_TRUNC('MONTH', order_date) AS Order_Month,
  SUM(total_price) AS Total_Revenue,
  SUM(total_price) FILTER (WHERE order_status='O') AS Total_Revenue_for_Open_Orders
FROM orders_demo
GROUP BY DATE_TRUNC('MONTH', order_date)
ORDER BY Order_Month;

In [0]:
DESCRIBE EXTENDED orders_metrics_mv;